In [6]:
import numpy as np
from flask import Flask, request, render_template
import pickle
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
#Creat an app object using the flask class
app = Flask(__name__)

In [8]:
# load the trained model. (pickle file)
try:
	with open('C:/Users/OMEN/Desktop/Flask/models/model.pkl', 'rb') as file:
		model = pickle.load(file)
except pickle.UnpicklingError:
	print("Error: The file 'models.pkl' is not a valid pickle file or is corrupted.")
except FileNotFoundError:
	print("Error: The file 'models.pkl' was not found. Please check the file path.")
except Exception as e:
	print(f"An unexpected error occurred: {e}")

In [9]:
#Define the route to be home 
#The decorator below links the relative route of the URL to the function it is dec
#Here, home function is with '/', our root directory.
# Running the app sends us to index.html.
# Note that the render_template means it looks for the file in the templates folder. 
#use the route() decorator to tell Flask what URL should trigger our function.
@app.route('/')
def home():
    return render_template('index.html')

In [10]:
# Ensure NLTK resources are available
try:
    nltk.download('punkt')
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))
    portStemmer = PorterStemmer()
except Exception as e:
    print(f"Error setting up NLTK resources: {e}")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\OMEN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\OMEN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
# Load the trained model and vectorizer
try:
    with open('C:/Users/OMEN/Desktop/Flask/models/model.pkl', 'rb') as file:
        model = pickle.load(file)
    
    # Load the same vectorizer that was used during training
    with open('C:/Users/OMEN/Desktop/Flask/models/vectorizer.pkl', 'rb') as file:
        vectorizer = pickle.load(file)
        
except pickle.UnpicklingError:
    print("Error: A pickle file is not valid or is corrupted.")
except FileNotFoundError as e:
    print(f"Error: File not found. {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Error: A pickle file is not valid or is corrupted.


In [12]:


# Text preprocessing functions
def clean(text: str) -> str:
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text

def preprocess(text: str):
    text = clean(text)
    text = text.split()
    tokens = [portStemmer.stem(word) for word in text if word not in stop_words]
    return ' '.join(tokens)

# Define the route to be home
@app.route('/')
def home():
    return render_template('index.html')

# Add Post method to the decorator to allow for form submission
@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Get the news article text from the form
        article_text = request.form['article']
        
        # Preprocess the input text
        processed_text = preprocess(article_text)
        
        # Since your model was trained on combined title and text,
        # but our form only has one input field, we'll treat the entire input as combined
        # Transform the text using the same vectorizer used during training
        features = vectorizer.transform([processed_text])
        
        # Make prediction
        prediction = model.predict(features)
        
        # Convert prediction to readable output
        # Adjust based on your specific label encoding
        if prediction[0] == 0:
            output = "FAKE"
        else:
            output = "REAL"
        
        return render_template('index.html', prediction_text='This news is {}'.format(output))
    
    except Exception as e:
        print(f"Error during prediction process: {e}")
        return render_template('index.html', prediction_text='Error processing your request')

if __name__ == "__main__":
    app.run(debug=True)

AssertionError: View function mapping is overwriting an existing endpoint function: home